In [7]:
import DBcm
import os
import swim_utils
import datetime

files = os.listdir(swim_utils.FOLDER)

files.remove(".DS_Store")
swimmer_data = []
event_data = []
time_data = []

config = {
    "user": "swimuser",
    "password": "swimuserpasswd",
    "database": "swimmer_database",
    "host": "localhost",
}

with open("ingest_data.ipynb") as f:
    data = f.read()

In [8]:
#Populate swimmers_data
for file in files:
    print(file)
    (
        name,
        age,
        distance,
        stroke,
        the_times,
        converts,
        the_average,
    ) = swim_utils.get_swimmers_data(file)
    if (name, age) in swimmer_data:
        continue
    else:
        swimmer_data.append((name,age))
        
swim_insert_SQL = """    
    INSERT IGNORE INTO swimmers
    (name, age)
    VALUES
    (%s, %s)
    """
        
with DBcm.UseDatabase(config) as db:
    db.executemany(swim_insert_SQL, swimmer_data)

Mike-15-100m-Fly.txt
Aurora-13-50m-Free.txt
Calvin-9-50m-Back.txt
Blake-15-100m-Fly.txt
Chris-17-100m-Breast.txt
Darius-13-100m-Fly.txt
Maria-9-50m-Free.txt
Bill-18-200m-Back.txt
Abi-10-50m-Breast.txt
Calvin-9-50m-Fly.txt
Ruth-13-200m-Back.txt
Mike-15-200m-Free.txt
Tasmin-15-100m-Breast.txt
Owen-15-100m-Free.txt
Chris-17-100m-Back.txt
Alison-14-100m-Breast.txt
Mike-15-100m-Back.txt
Darius-13-100m-Back.txt
Emma-13-100m-Breast.txt
Tasmin-15-100m-Back.txt
Lizzie-14-100m-Back.txt
Bill-18-100m-Back.txt
Tasmin-15-100m-Free.txt
Elba-14-100m-Free.txt
Lizzie-14-100m-Free.txt
Ruth-13-200m-Free.txt
Ali-12-100m-Back.txt
Darius-13-200m-IM.txt
Katie-9-50m-Back.txt
Katie-9-100m-Breast.txt
Hannah-13-100m-Back.txt
Katie-9-50m-Free.txt
Katie-9-100m-Back.txt
Erika-15-100m-Breast.txt
Mike-15-100m-Free.txt
Hannah-13-100m-Free.txt
Blake-15-100m-Back.txt
Katie-9-50m-Fly.txt
Abi-10-100m-Breast.txt
Calvin-9-50m-Free.txt
Erika-15-100m-Free.txt
Erika-15-200m-Breast.txt
Darius-13-100m-Breast.txt
Katie-9-100m-Free

In [9]:
# Populate event_data        
for file in files:
    (
        name,
        age,
        distance,
        stroke,
        the_times,
        converts,
        the_average,
    ) = swim_utils.get_swimmers_data(file)
    if (distance, stroke) in event_data:
        continue
    else:
        event_data.append((distance,stroke))
        
event_insert_SQL = """
    INSERT IGNORE INTO events 
    (distance, event)
    VALUES
    (%s, %s)
    """
        
with DBcm.UseDatabase(config) as db:
    db.executemany(event_insert_SQL, event_data)

In [10]:
swim_select_SQL = """
    SELECT swimmer_id 
    FROM swimmers 
    WHERE name = %s AND age = %s;"""

swimmer_ids = []

for swimmer in files:
    name, age = swim_utils.get_swimmers_data(swimmer)[:2]

    with DBcm.UseDatabase(config) as db:
        db.execute(swim_select_SQL, (name, age))
        swimmer_ids.append(db.fetchone())

print(swimmer_ids)

InternalError: Unread result found

In [ ]:
event_select_SQL = """
    SELECT event_id 
    FROM events 
    WHERE distance = %s AND event = %s;"""
    
event_ids = []
for event in files:
    
    distance, stroke = swim_utils.get_swimmers_data(event)[2:4]
    
    with DBcm.UseDatabase(config) as db:
        db.execute(event_select_SQL, (distance, stroke))
        event_ids.append(db.fetchone())
            
print(event_ids)

[(1,), (2,), (3,), (1,), (4,), (1,), (2,), (5,), (6,), (7,), (5,), (8,), (4,), (9,), (10,), (4,), (10,), (10,), (4,), (10,), (10,), (10,), (9,), (9,), (9,), (8,), (10,), (11,), (3,), (4,), (10,), (2,), (10,), (4,), (9,), (9,), (10,), (7,), (4,), (2,), (9,), (12,), (4,), (9,), (10,), (9,), (9,), (9,), (10,), (5,), (2,), (9,), (9,), (6,), (3,), (13,), (9,), (10,), (12,), (11,)]


In [ ]:
ts = datetime.datetime.now()

current = 0
for file in files:
    (
        name,
        age,
        distance,
        stroke,
        the_times,
        converts,
        the_average,
    ) = swim_utils.get_swimmers_data(file)

    with DBcm.UseDatabase(config) as db:
        for time in the_times:
            insert_times_SQL = f"""
            INSERT INTO times (swimmer_id, event_id, times, ts) 
            VALUES ({swimmer_ids[current][0]}, {event_ids[current][0]}, "{time}", "{ts}");
            """
            db.execute(insert_times_SQL)
    current += 1
        
print(swimmer_data)
print(event_data)

[('Mike', '15'), ('Aurora', '13'), ('Calvin', '9'), ('Blake', '15'), ('Chris', '17'), ('Darius', '13'), ('Maria', '9'), ('Bill', '18'), ('Abi', '10'), ('Ruth', '13'), ('Tasmin', '15'), ('Owen', '15'), ('Alison', '14'), ('Emma', '13'), ('Lizzie', '14'), ('Elba', '14'), ('Ali', '12'), ('Katie', '9'), ('Hannah', '13'), ('Erika', '15'), ('Carl', '15'), ('Dave', '17')]
[('100m', 'Fly'), ('50m', 'Free'), ('50m', 'Back'), ('100m', 'Breast'), ('200m', 'Back'), ('50m', 'Breast'), ('50m', 'Fly'), ('200m', 'Free'), ('100m', 'Free'), ('100m', 'Back'), ('200m', 'IM'), ('200m', 'Breast'), ('400m', 'Free')]
